In [7]:
import numpy as np

import mne

from mne.preprocessing import ICA

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict, LeaveOneOut, LeavePOut

import matplotlib.pyplot 

# Chemin vers les fichiers EEG et les positions des électrodes
raw_fnames = 'C:/recordings/Game_recordings_test/TOUT_4_second_apple/GAMING_APPLE_1_noc.set'
montage_fname = 'path/to/montage_file.elc'  # Remplacez par le chemin vers votre fichier de montage

# Charger les données EEG
raw = mne.io.read_raw_eeglab(raw_fnames, preload=True)

# Charger et définir le montage
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)
raw.set_eeg_reference('average', projection=True)



Reading C:\recordings\Game_recordings_test\TOUT_4_second_apple\GAMING_APPLE_1_noc.fdt
Reading 0 ... 42499  =      0.000 ...   169.996 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [8]:
new_srate = 25
tmin = -0.5  # début de chaque époque (1,0 secondes avant le déclencheur)
tmax = 1.5  # fin de chaque époque (au déclencheur)
baseline = (-0.5, 0)  # baseline du début à t = 0
reject = dict(eeg=150e-3)  # seuil de rejet pour les électrodes EEG
L_H_freq = [(8,13),(13,30)]
Notchs_freq = (50,60)
f_low = 1
raw.filter( f_low, None)
raw.notch_filter(Notchs_freq)
raw.resample(new_srate)

events , event_id= mne.events_from_annotations(raw)
event_id_RL= {'right' : event_id['right'] , 'left' : event_id['left']}
epochs = mne.Epochs(raw, events, event_id_RL, tmin, tmax, proj=True, baseline=baseline, reject=reject, preload=True)
info = epochs.info

# Calculer la matrice de covariance de bruit
noise_cov = mne.make_ad_hoc_cov(info)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 825 samples (3.300 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Used Annotations descriptions: ['STOP', 'left', 'right']
Not setting metadata
41 matching events found
Applying baseline correction (mo

In [9]:
# Utiliser un modèle de tête standard fourni par MNE
subjects_dir = 'C:/Users/robinaki/mne_data/MNE-sample-data/subjects'


# noise_cov = mne.make_ad_hoc_cov(info)
mne.datasets.fetch_fsaverage(subjects_dir=subjects_dir)
bem = mne.read_bem_solution(
'C:/Users/robinaki/mne_data/MNE-sample-data/subjects/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif'
    )
src = mne.setup_source_space("fsaverage", spacing="oct6", add_dist="patch", subjects_dir=subjects_dir)
fwd = mne.make_forward_solution(
        info,
        trans="fsaverage",
        src=src,
        bem=bem,
        meg=False,
        eeg=True,
        mindist=5.0,
        n_jobs=None,
        verbose=False,
    )





0 files missing from root.txt in C:\Users\robinaki\mne_data\MNE-sample-data\subjects
0 files missing from bem.txt in C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage
Loading surfaces...

Loading the solution matrix...

Three-layer model surfaces loaded.
Loaded linear collocation BEM solution from C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Setting up the source space with the following parameters:

SUBJECTS_DIR = C:\Users\robinaki\mne_data\MNE-sample-data\subjects
Subject      = fsaverage
Surface      = white
Octahedron subdivision grade 6

>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage\surf\lh.white...
Mapping lh fsaverage -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from C:\Users\robinaki\mne_data\MNE-sample-data\subjects\fsaverage\surf\lh.sphere...
Setting up the triangulation for th

In [12]:

# Calculer la solution inverse

inverse_operator = mne.minimum_norm.make_inverse_operator(info, fwd, noise_cov, loose=0.2, depth=0.8)
# stc = mne.minimum_norm.apply_inverse_raw(raw, inverse_operator, lambda2=1.0 / 9.0, method='dSPM')

# Solution of the inverse solution for evoked data (averaged data) by marker (event ID left and right)

event_id_R =  {'right' : event_id['right']}
event_id_L= {'left' : event_id['left']}
epochsR =  mne.Epochs(raw, events, event_id_R, tmin, tmax, proj=True, baseline=baseline, reject=reject, preload=True)
epochsL =  mne.Epochs(raw, events, event_id_L, tmin, tmax, proj=True, baseline=baseline, reject=reject, preload=True)

evokedL = epochsL.average()
evokedR = epochsL.average()

stcR, residualR = mne.minimum_norm.apply_inverse(
    evokedR,
    inverse_operator,
    lambda2=1.0 / 9.0,
    method='dSPM',
    pick_ori=None,
    return_residual=True,
    verbose=True,
)
stcL, residualR = mne.minimum_norm.apply_inverse(
    evokedL,
    inverse_operator,
    lambda2=1.0 / 9.0,
    method='dSPM',
    pick_ori=None,
    return_residual=True,
    verbose=True,
)
stc= mne.minimum_norm.apply_inverse_raw(
    raw,
    inverse_operator,
    lambda2=1.0 / 9.0,
    method='dSPM',
    pick_ori=None,
    verbose=True,
)

Converting forward solution to surface orientation
    Average patch normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 8 channels.
    8 out of 8 channels remain after picking
Selected 8 channels
Creating the depth weighting matrix...
    8 EEG channels
    limit = 8197/8196 = 5.326282
    scale = 7695.39 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 7.1e-19 (2.2e-16 eps * 8 dim * 0.0004  max singular value)
    Estimated rank (eeg): 7
    EEG: rank 7 computed from 8 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.
   

In [13]:

# Visualiser les résultats
brain = stc.plot(hemi='lh', subjects_dir=subjects_dir, initial_time=0.1, time_viewer=True)

Using control points [ 50.0526319   55.47275608 147.4279748 ]


Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/743.8 kB ? eta -:--:--
   - ------------------------------------- 30.7/743.8 kB 660.6 kB/s eta 0:00:02
   -------- ------------------------------- 153.6/743.8 kB 1.8 MB/s eta 0:00:01
   ------------------------ --------------- 450.6/743.8 kB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 743.8/743.8 kB 5.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
